# Korean LLM (Large Language Model) Inference
---

### Model: [KoAlpaca-Polyglot-12.8B](https://huggingface.co/beomi/KoAlpaca-Polyglot-12.8B)

- GitHub: https://github.com/Beomi/KoAlpaca

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates') 

In [2]:
!pip install -qU boto3 huggingface_hub sagemaker langchain deepspeed 
!pip install -qU bitsandbytes accelerate peft

In [4]:
import os
from pathlib import Path
from huggingface_hub import snapshot_download

HF_MODEL_ID = "beomi/KoAlpaca-Polyglot-12.8B"

# Only download pytorch checkpoint files
#allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]
allow_patterns = ["*.json", "*.pt", "*.txt", "*.model", "*.safetensors"]

# create model dir
model_name = HF_MODEL_ID.split("/")[-1].replace('.', '-')
model_tar_dir = Path(f"/home/ec2-user/SageMaker/models/{model_name}")
if not os.path.isdir(model_tar_dir):
    os.makedirs(model_tar_dir, exist_ok=True)
    # Download model from Hugging Face into model_dir
    snapshot_download(
        HF_MODEL_ID, 
        local_dir=str(model_tar_dir), 
        local_dir_use_symlinks=False,
        allow_patterns=allow_patterns,
        cache_dir="/home/ec2-user/SageMaker/"
    )

Fetching 37 files:   0%|          | 0/37 [00:00<?, ?it/s]

In [5]:
import torch
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
import torch
import deepspeed
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, GPTNeoXLayer

model = AutoModelForCausalLM.from_pretrained(
    model_tar_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    #cache_dir=local_model_path,
    quantization_config=bnb_config
)

[2023-07-30 07:34:26,475] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [7]:
# config = {
#     "tensor_parallel": {"tp_size": 1},
#     "dtype": "fp16",
#     "injection_policy": {
#         GPTNeoXLayer:('attention.dense', 'mlp.dense_4h_to_h')
#     }
# }

# modl = deepspeed.init_inference(model, config)

local_rank = int(os.getenv('LOCAL_RANK', '0'))
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_ID)
generator = pipeline(
    task="text-generation", model=model, tokenizer=tokenizer
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [8]:
import json
from inference_lib import Prompter

prompter = Prompter("kullm")

params = {
    "do_sample": False,
    "max_new_tokens": 256,
    "return_full_text": True,
    "temperature": 0.2,
    "top_p": 0.9,
    "return_full_text": False,
    "repetition_penalty": 1.1,
    "presence_penalty": None,
    "eos_token_id": 2,
}

instruction = "아래 질문에 대답해줘."
#instruction = ""
input_text = "아마존 웹서비스(AWS)에 대해 알려줘"
prompt = prompter.generate_prompt(instruction, input_text)
payload = {
    "inputs": [prompt,],
    "parameters": params
}

In [9]:
text_inputs, params = payload["inputs"], payload["parameters"]
result = generator(text_inputs, **params)
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[[{'generated_text': 'AWS은 클라우드 컴퓨팅 서비스로, EC2 인스턴스, RDS, Elastic Beanstalk, S3 등의 다양한 서비스를 제공하고 있습니다.'}]]
